In [28]:
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict
import re
from typing import List, Dict
from typing import List, TypedDict

from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.messages import BaseMessage

from langgraph.graph.message import add_messages

from pydantic import BaseModel, Field


from langgraph.prebuilt import tools_condition

In [29]:
import getpass
import os


def _set_env(key: str):
    if key not in os.environ:
        os.environ[key] = getpass.getpass(f"{key}:")


_set_env("OPENAI_API_KEY")

In [31]:
from typing import TypedDict, Sequence, Literal, List, Dict, Any
from langchain.schema import BaseMessage, HumanMessage

import pprint
import re

# Update AgentState to include required fields
class AgentState(TypedDict):
    messages: List[Any]
    classes: List[str]
    blocks: List[str]
    extracted_rubric: Dict[str, str]
    graded_responses: List[Dict[str, str]]
    marks: List[str]
    final_score: str

def extract_classes(input_string):
    print(input_string)
    
    class_pattern = r'List of classes\[(.*?)\]'
    # Find the list of classes using regex
    class_match = re.search(class_pattern, input_string)
    if class_match:
        # Extract the classes as a string and convert to a list by splitting
        class_list_str = class_match.group(1)
        # Removing extra spaces and splitting the string into a list
        class_list = [cls.strip() for cls in class_list_str.split(',')]
    else:
        class_list = []
    
    # Split the code blocks by the delimiter '________end________'
    code_blocks = input_string.split('________end________')
    
    # Remove any trailing or empty lines from each code block
    code_blocks = [code.strip() for code in code_blocks if code.strip()]
    
    return class_list, code_blocks

# Example functions

def extract(state: AgentState) -> AgentState:
    """
    Transform the query to produce a better question.

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with re-phrased question
    """

    print("---TRANSFORM QUERY---")
    messages = state["messages"]
    with open('simple-scenario/student_solution.md', 'r') as file:
        code = file.read()
    print(code)

    msg = [
        HumanMessage(
            content=f""" \n 
You are given a code snippet written in Java. 
Extract all class definitions from the code. 
For each class, provide the class name, code. 
Separate the code of each class with the line ________end________. 
At the end, provide a list of all the extracted class names.
Output Format:
<Insert class1 code here>
________end________
<Insert class1 code here>
________end________
List of classes["class1","class2"...etc]


    \n ------- \n
    {code} 
    \n ------- \n
    Do the task as directed""",
        )
    ]

    # Grader
    model = ChatOpenAI(temperature=0.7, model="gpt-4o-mini", streaming=True)
    response = model.invoke(msg)
    
    class_list, code_blocks = extract_classes(response.content)
    state["classes"] = class_list
    state["blocks"] = code_blocks
    state["messages"] = [response]
    
    return state

def extrcat_rubric(state: AgentState) -> AgentState:
    print("---EXTRACT RUBRIC---")
    # Read rubric.md
    with open('simple-scenario/rubric.md', 'r') as file:
        rubric = file.read()
    
    # Classes to be used for extraction
    print(state.keys())
    class_list = state["classes"]
    
    # Prompt to get the relevant parts of the rubric for each class
    model = ChatOpenAI(temperature=0.3, model="gpt-4o-mini", streaming=True)
    extracted_rubric = {}
    
    for class_name in class_list:
        prompt = f"""
        You are provided with the following rubric:
        {rubric}
        
        Extract the relevant parts of the rubric that apply to the following class: {class_name}.
        The relevant rubric for the class should be extracted and summarized accordingly.
        """
        response = model.invoke([prompt])
        extracted_rubric[class_name] = response.content
    
    # Updating the state with the extracted rubric information
    state["extracted_rubric"] = extracted_rubric
    return state

def grade(state: AgentState) -> AgentState:
    print("---GRADE CODE BLOCKS---")
    # Extracting rubric and code blocks from state
    extracted_rubric = state.get("extracted_rubric", {})
    code_blocks = state["blocks"]
    class_list = state["classes"]
    
    # Read the model solution from file
    with open('simple-scenario/model_solution.md', 'r') as file:
        model_solution = file.read()
    
    # Grading prompt for each code block
    model = ChatOpenAI(temperature=0.4, model="gpt-4o-mini", streaming=True)
    graded_responses = []
    
    for class_name, code_block in zip(class_list, code_blocks):
        rubric = extracted_rubric.get(class_name, "")
        prompt = f"""
        You are given a code block and a rubric for evaluation. Compare the code block with the model solution provided.
        
        Code Block:
        {code_block}
        
        Model Solution:
        {model_solution}
        
        Rubric for Evaluation:
        {rubric}
        
        Provide a grade and feedback for the code block as per the rubric.
        """
        response = model.invoke([prompt])
        graded_responses.append({"class": class_name, "feedback": response.content})
    
    # Update the state with graded responses
    state["graded_responses"] = graded_responses
    for key, value in state.items():
        print(f"{key}: {value}")
    return state

def extract_marks(state: AgentState) -> AgentState:
    print("---EXTRACT MARKS FROM FEEDBACK---")
    graded_responses = state.get("graded_responses", [])
    model = ChatOpenAI(temperature=0.3, model="gpt-4o-mini", streaming=True)
    marks = []
    
    prompt = """
    You are given a list of feedback strings. For each feedback, extract the marks awarded. 
    The marks will be provided in the format: Marks:[<number>].
    Extract the marks and provide them in a list format.
    Feedback List:
    {feedback_list}
    Output Format:
    Marks: [<mark1>, <mark2>, ...]
    """

    feedback_list = [response["feedback"] for response in graded_responses]
    response = model.invoke([HumanMessage(content=prompt.format(feedback_list=feedback_list))])
    
    # Extracting marks from response
    marks_match = re.search(r'Marks:\s*\[(.*?)\]', response.content)
    if marks_match:
        marks_str = marks_match.group(1)
        marks = [mark.strip() for mark in marks_str.split(',')]
    
    # Update the state with extracted marks
    state["marks"] = marks
    for key, value in state.items():
        print(f"{key}: {value}")
    return state

def sum_using_tool(state: AgentState) -> AgentState:
    print("---SUM MARKS USING LLM TOOL---")
    marks = state.get("marks", [])
    model = ChatOpenAI(temperature=0.3, model="gpt-4o-mini", streaming=True)
    
    prompt = """
    You are given a list of marks in string format. Each mark is represented as 'Marks:[<number>]'.
    Extract the numerical value from each string, sum them, and provide the final score.
    Marks List:
    {marks_list}
    Output Format:
    Final Score: [<total_marks>]
    """
    response = model.invoke([HumanMessage(content=prompt.format(marks_list=marks))])
    
    # Extracting final score from response
    score_match = re.search(r'Final Score:\s*\[(\d+)]', response.content)
    final_score = score_match.group(1) if score_match else "0"
    
    # Update the state with the final score
    state["final_score"] = final_score
    print(f"Final Score: {final_score}")
    return state

from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode

# Define a new graph
workflow = StateGraph(AgentState)

# Define the nodes we will cycle between
workflow.add_node("extract", extract)  # agent
workflow.add_node("rubric", extrcat_rubric)
workflow.add_node("grader", grade)
workflow.add_node("extract_marks", extract_marks)
workflow.add_node("sum_using_tool", sum_using_tool)

workflow.add_edge(START, "extract")
workflow.add_edge("extract", "rubric")
workflow.add_edge("rubric", "grader")
workflow.add_edge("grader", "extract_marks")
workflow.add_edge("extract_marks", "sum_using_tool")
workflow.add_edge("sum_using_tool", END)

# Compile
graph = workflow.compile()

# Stream output
user_input = "input"
inputs = {
    "messages": [],
    "classes": [],
    "blocks": [],
    "extracted_rubric": {},
    "graded_responses": [],
    "marks": [],
    "final_score": ""
}

for output in graph.stream(inputs):
    for key, value in output.items():
        pprint.pprint(f"Output from node '{key}':")
        pprint.pprint("---")
        pprint.pprint(value.get('messages', []), indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")

---TRANSFORM QUERY---
```java
import java.util.Scanner;

public class StringManipulator {
    public static void main(String[] args) {
        Scanner sc = new Scanner(System.in);
        System.out.print("Enter a string: ");
        String input = sc.next();

        System.out.println("Original String: " + input);
        System.out.println("Uppercase String: " + input.toLowerCase());

        String reversed = "";
        for (int i = 0; i <= input.length(); i++) { 
            reversed += input.charAt(i); 
        }
        System.out.println("Reversed String: " + reversed);

        System.out.println("Number of Characters: " + (input.length() - 1)); 
    }
}
```

```java
public class StringManipulator {
    public static void main(String[] args) {
        Scanner sc = new Scanner(System.in);
        System.out.print("Enter a string: ");
        String input = sc.next();

        System.out.println("Original String: " + input);
        System.out.println("Uppercase String: " + inp